# 📝 AutoPrompt with Llama 3.2 (Open Source)

This notebook demonstrates how to run **AutoPrompt** using the open-source **unsloth/Llama-3.2-3B-Instruct** model instead of the OpenAI API.

**Requirements:**
- Google Colab with GPU runtime (T4 or better)
- No API keys needed!

**Setup:** Go to `Runtime` → `Change runtime type` → Select **T4 GPU**

## 1. Install Dependencies

In [ ]:
!git clone https://github.com/joysontelem/AutoPrompt.git
%cd AutoPrompt
!pip install -r requirements.txt -q

## 2. Verify GPU Availability

In [ ]:
import torch
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")

## 3. Quick Test – Load the Llama 3.2 Model

Verify the model loads correctly before running the full pipeline.

In [ ]:
from utils.config import get_llm

llm_config = {
    'type': 'HuggingFacePipeline',
    'name': 'unsloth/Llama-3.2-3B-Instruct',
    'chat_model': True,
    'max_new_tokens': 256,
    'device_map': 'auto',
    'temperature': 0.7
}

llm = get_llm(llm_config)
print("Model loaded successfully!")
print(type(llm))

## 4. Run the Classification Pipeline

Run AutoPrompt with Llama 3.2 for a classification task. This uses the `config/config_llama.yml` configuration.

In [ ]:
!python run_pipeline.py \
    --basic_config_path config/config_llama.yml \
    --prompt "Does this movie review contain a spoiler? answer Yes or No" \
    --task_description "Classify whether a movie review contains spoilers." \
    --num_steps 5

## 5. Run Benchmark Optimization (Optional)

If you have a pre-labeled CSV dataset with `text` and `annotation` columns, you can run the benchmark optimization mode.

In [ ]:
# Example: Create a small sample dataset
import pandas as pd

data = {
    'text': [
        'The movie was absolutely fantastic! Great acting and storyline.',
        'Terrible film. Waste of time and money.',
        'An average movie with some good moments but overall forgettable.',
        'One of the best movies I have ever seen. Highly recommend!',
        'The plot was confusing and the ending made no sense.',
        'Beautiful cinematography and a touching story.',
        'I fell asleep halfway through. So boring.',
        'A masterpiece of modern cinema.',
        'Not worth the ticket price.',
        'Enjoyable family movie with great humor.'
    ],
    'annotation': ['Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes']
}

df = pd.DataFrame(data)
df.to_csv('sample_benchmark.csv', index=False)
print(f'Created sample dataset with {len(df)} samples')
df.head()

In [ ]:
!python run_benchmark_optimization.py \
    --config config/config_llama.yml \
    --dataset sample_benchmark.csv \
    --prompt "Is this movie review positive? Answer Yes or No." \
    --task_description "Classify movie reviews as positive or negative." \
    --labels Yes No \
    --num_steps 3 \
    --output llama_results.json

In [ ]:
# View the results
import json

try:
    with open('llama_results.json', 'r') as f:
        results = json.load(f)
    print(f"Initial Score: {results['initial_score']:.2%}")
    print(f"Best Score: {results['best_score']:.2%}")
    print(f"Iterations: {results['num_iterations']}")
    print(f"\nBest Prompt:\n{results['best_prompt']}")
except FileNotFoundError:
    print("Results file not found. Run the benchmark optimization cell first.")